## Step 0: Import Packages and Mount Drive

In [0]:
# Install Packages
!pip install requests
!pip install bs4
!pip install progressbar

# Import Packages
import pandas as pd
import re
import requests
import bs4
from bs4 import BeautifulSoup
import os, os.path, csv
from progressbar import ProgressBar


In [0]:
# Mount User's Drive
# Copy and paste the passkey from your Google account
# You should use the same account that is operating the Colab file
# Ignore if you aren't accessing this file in Google Colab

#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [0]:
#parent_dir_name = 'GxE with GEE'

#for dirpath, subdirs, files in os.walk('/content'):
#  if parent_dir_name in subdirs:
#    parent_path = dirpath + "/" + parent_dir_name

#parent_path

In [0]:
# Set Data Path
## Change the destination to your Drive directory containing the folder 'raw_data.zip'
#data_path_end = '/Colab Workspace/Data'
#os.chdir(parent_path + data_path_end)

# Print the current working directory
#!pwd

## Step 1: Make a List of All CIMMYT Wheat Germplasm Bank URLs

In [0]:
max_varieties = 180000
base_url = 'http://wgb.cimmyt.org/gringlobal/accessiondetail.aspx?id='
url_list = list(range(1, max_varieties))
url_list = pd.DataFrame(url_list, columns=['number'])
url_list['url'] = base_url + url_list['number'].astype(str)
url_list.drop('number',axis=1,inplace=True)
url_list.loc[0][0]

'http://wgb.cimmyt.org/gringlobal/accessiondetail.aspx?id=1'

## Step 2: Example on a Single Page

Tutorial Source: https://medium.com/@kashaziz/scrap-a-web-page-in-20-lines-of-code-with-python-and-beautifulsoup-b95c58e93124


In [0]:
response = requests.get(url_list.loc[97148][0])
soup = BeautifulSoup(response.text,'html.parser')

In [0]:
soup.head()

[<link href="/gringlobal/images/sysblank.ico" rel="shortcut icon"/>,
 <link href="/gringlobal/styles/default.aspx?theme=theme1" id="ctl00_lnkTheme" media="screen" rel="stylesheet"/>,
 <link href="/gringlobal/styles/print.css" media="print" rel="stylesheet" type="text/css"/>,
 <title>
 	Accession: BW 3816  - GRIN-Global Web v 1.10.4.1
 </title>,
 <script src="/gringlobal/scripts/jquery-1.3.2.min.js" type="text/javascript"></script>,
 <script src="/gringlobal/scripts/jquery.cookie.js" type="text/javascript"></script>,
 <script type="text/javascript">
         $(document).ready(function() {
             $('._blurme').blur();
             $('.btn').each(function() {
                 var htmlStr = $(this).html();
                 if (htmlStr.indexOf("<i>") == -1) {
                     $(this).append('<i>').append('<span>').wrapInner(document.createElement("SPAN")).prepend('<i>');
                 }
             });
         });
     </script>,
 <script type="text/javascript">
     $(docume

In [0]:
GID_CID_SID = pd.DataFrame(columns=['gid','cid_sid'])

response = requests.get(url_list.loc[97150-1][0])
soup = BeautifulSoup(response.text,'html.parser')
main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
table_headings = main_content.find_all('th')
variety_info = pd.DataFrame(columns=['gid','cid_sid'])
for result in table_headings:
    if 'GID' in result.text:
        variety_info.loc[0,'gid'] = result.text
    if 'CID' in result.text:
        variety_info.loc[0,'cid_sid'] = result.text

variety_info

,gid,cid_sid
0,GID 2155737,CID 266244 SID 11


In [0]:
GID_CID_SID = GID_CID_SID.append(variety_info)
GID_CID_SID

,gid,cid_sid
0,GID 2155741,CID 266244 SID 9


## Step 3: Loop over First 30000 URLS
Took roughly 10 hours

In [0]:
GID_CID_SID = pd.DataFrame(columns=['url_num','gid','cid_sid'])

pbar = ProgressBar()

for i in pbar(range(0, 30000)) :
    try:
        response = requests.get(url_list.loc[i][0])
        soup = BeautifulSoup(response.text,'html.parser')
        main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
        table_headings = main_content.find_all('th')
        variety_info = pd.DataFrame(columns=['url_num','gid','cid_sid'])
        variety_info['url_num'] = [i+1]
        for result in table_headings:
            if 'GID' in result.text:
                variety_info.loc[0,'gid'] = result.text
            if 'CID' in result.text:
                variety_info.loc[0,'cid_sid'] = result.text

        GID_CID_SID = GID_CID_SID.append(variety_info)
    except:
        print('error processing url number {}'.format(i+1))

error processing url number 5
error processing url number 6
error processing url number 51
error processing url number 87
error processing url number 272


error processing url number 445


error processing url number 614
error processing url number 615


error processing url number 1101


error processing url number 1399


...

error processing url number 20721
error processing url number 20722


error processing url number 24932


error processing url number 26418


error processing url number 29550


100% |########################################################################|


In [0]:
GID_CID_SID.to_csv('GID_CID_SID_30k.csv')

## Run Loop for Rest of Observations

In [0]:
GID_CID_SID_60k = pd.DataFrame(columns=['url_num','gid','cid_sid'])

pbar = ProgressBar()

for i in pbar(range(30000, 60000)) :
    try:
        response = requests.get(url_list.loc[i][0])
        soup = BeautifulSoup(response.text,'html.parser')
        main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
        table_headings = main_content.find_all('th')
        variety_info = pd.DataFrame(columns=['url_num','gid','cid_sid'])
        variety_info['url_num'] = [i+1]
        for result in table_headings:
            if 'GID' in result.text:
                variety_info.loc[0,'gid'] = result.text
            if 'CID' in result.text:
                variety_info.loc[0,'cid_sid'] = result.text

        GID_CID_SID_60k = GID_CID_SID_60k.append(variety_info)
    except:
        print('error processing url number {}'.format(i+1))

In [0]:
GID_CID_SID_60k.to_csv('GID_CID_SID_60k.csv')

In [0]:
GID_CID_SID_90k = pd.DataFrame(columns=['url_num','gid','cid_sid'])

pbar = ProgressBar()

for i in pbar(range(60000, 90000)) :
    try:
        response = requests.get(url_list.loc[i][0])
        soup = BeautifulSoup(response.text,'html.parser')
        main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
        table_headings = main_content.find_all('th')
        variety_info = pd.DataFrame(columns=['url_num','gid','cid_sid'])
        variety_info['url_num'] = [i+1]
        for result in table_headings:
            if 'GID' in result.text:
                variety_info.loc[0,'gid'] = result.text
            if 'CID' in result.text:
                variety_info.loc[0,'cid_sid'] = result.text

        GID_CID_SID_90k = GID_CID_SID_90k.append(variety_info)
    except:
        print('error processing url number {}'.format(i+1))

In [0]:
GID_CID_SID_90k.to_csv('GID_CID_SID_90k.csv')

In [0]:
GID_CID_SID_120k = pd.DataFrame(columns=['url_num','gid','cid_sid'])

pbar = ProgressBar()

for i in pbar(range(90000, 120000)) :
    try:
        response = requests.get(url_list.loc[i][0])
        soup = BeautifulSoup(response.text,'html.parser')
        main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
        table_headings = main_content.find_all('th')
        variety_info = pd.DataFrame(columns=['url_num','gid','cid_sid'])
        variety_info['url_num'] = [i+1]
        for result in table_headings:
            if 'GID' in result.text:
                variety_info.loc[0,'gid'] = result.text
            if 'CID' in result.text:
                variety_info.loc[0,'cid_sid'] = result.text

        GID_CID_SID_120k = GID_CID_SID_120k.append(variety_info)
    except:
        print('error processing url number {}'.format(i+1))

In [0]:
GID_CID_SID_120k.to_csv('GID_CID_SID_120k.csv')

NameError: name 'GID_CID_SID_120k' is not defined

In [0]:
GID_CID_SID_150k = pd.DataFrame(columns=['url_num','gid','cid_sid'])

pbar = ProgressBar()

for i in pbar(range(120000, 150000)) :
    try:
        response = requests.get(url_list.loc[i][0])
        soup = BeautifulSoup(response.text,'html.parser')
        main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
        table_headings = main_content.find_all('th')
        variety_info = pd.DataFrame(columns=['url_num','gid','cid_sid'])
        variety_info['url_num'] = [i+1]
        for result in table_headings:
            if 'GID' in result.text:
                variety_info.loc[0,'gid'] = result.text
            if 'CID' in result.text:
                variety_info.loc[0,'cid_sid'] = result.text

        GID_CID_SID_150k = GID_CID_SID_150k.append(variety_info)
    except:
        print('error processing url number {}'.format(i+1))

In [0]:
GID_CID_SID_150k.to_csv('GID_CID_SID_150k.csv')

In [0]:
GID_CID_SID_180k = pd.DataFrame(columns=['url_num','gid','cid_sid'])

pbar = ProgressBar()

for i in pbar(range(150000, 180000)) :
    try:
        response = requests.get(url_list.loc[i][0])
        soup = BeautifulSoup(response.text,'html.parser')
        main_content = soup.find('div', attrs = {'id': 'ctl00_cphBody_plAccessionNames'})
        table_headings = main_content.find_all('th')
        variety_info = pd.DataFrame(columns=['url_num','gid','cid_sid'])
        variety_info['url_num'] = [i+1]
        for result in table_headings:
            if 'GID' in result.text:
                variety_info.loc[0,'gid'] = result.text
            if 'CID' in result.text:
                variety_info.loc[0,'cid_sid'] = result.text

        GID_CID_SID_180k = GID_CID_SID_180k.append(variety_info)
    except:
        print('error processing url number {}'.format(i+1))

In [0]:
GID_CID_SID_180k.to_csv('GID_CID_SID_180k.csv')

## Append all Scraped Data to One Frame

In [0]:
GID_CID_SID = pd.read_csv('GID_CID_SID_30k.csv')
GID_CID_SID_60k = pd.read_csv('GID_CID_SID_60k.csv')
GID_CID_SID_90k = pd.read_csv('GID_CID_SID_90k.csv')
GID_CID_SID_120k = pd.read_csv('GID_CID_SID_120k.csv')
GID_CID_SID_150k = pd.read_csv('GID_CID_SID_150k.csv')
GID_CID_SID_180k = pd.read_csv('GID_CID_SID_180k.csv')


In [0]:
temp1 = GID_CID_SID.append(GID_CID_SID_60k)

In [0]:
temp2 = temp1.append(GID_CID_SID_90k)

In [0]:
temp3 = temp2.append(GID_CID_SID_120k)

In [0]:
temp4 = temp3.append(GID_CID_SID_150k)

In [0]:
GID_CID_SID_all = temp4.append(GID_CID_SID_180k)

In [0]:
GID_CID_SID_all.to_csv('GID_CID_SID_all.csv')